In [47]:
import pandas as pd
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
import matplotlib as mpl
import numpy as np
import seaborn as sns
from typing import List

In [27]:
red_wines = pd.read_csv("dataset/winequality-red.csv", sep=";")
white_wines = pd.read_csv("dataset/winequality-white.csv", sep=";")

In [28]:
red_wines["wine_type"] = "red"
white_wines["wine_type"] = "white"

In [29]:
def encode_quality(quality):
    if quality < 5:
        return "low"
    elif quality < 7:
        return "medium"
    return "high"


red_wines["quality_label"] = pd.Categorical(red_wines["quality"].apply(encode_quality), categories=["low", "medium", "high"], ordered=True)
white_wines["quality_label"] = pd.Categorical(white_wines["quality"].apply(encode_quality), categories=["low", "medium", "high"], ordered=True)

In [30]:
wines = pd.concat([red_wines, white_wines], ignore_index=True)
wines = wines.sample(frac=1, random_state=42)
wines.head()

,fixed acidity,volatile acidity,citric acid,residual sugar,chlorides,free sulfur dioxide,total sulfur dioxide,density,pH,sulphates,alcohol,quality,wine_type,quality_label
3103,7.0,0.17,0.74,12.8,0.045,24.0,126.0,0.99420,3.26,0.38,12.2,8,white,high
1419,7.7,0.64,0.21,2.2,0.077,32.0,133.0,0.99560,3.27,0.45,9.9,5,red,medium
4761,6.8,0.39,0.34,7.4,0.020,38.0,133.0,0.99212,3.18,0.44,12.0,7,white,high
4690,6.3,0.28,0.47,11.2,0.040,61.0,183.0,0.99592,3.12,0.51,9.5,6,white,medium
4032,7.4,0.35,0.20,13.9,0.054,63.0,229.0,0.99888,3.11,0.50,8.9,6,white,medium


In [54]:
def concat_described(data_frames: List[pd.DataFrame], subset_attributes : List[str], keys: List[str], drop_count = True):
    assert len(data_frames) == len(keys), "Number of data frames and keys should be equal"
    
    described_dfs = []
    for df in data_frames:
        described_dfs.append(df[subset_attributes].describe().drop("count") if drop_count else df[subset_attributes].describe())
        
    concatenated_df = pd.DataFrame()
    
    for i, col in enumerate(described_dfs[0].columns):
        col_comparison = pd.concat([df[col] for df in described_dfs], axis=1)
        col_comparison.columns = [f"{key} {col}" for key in keys]
        concatenated_df = pd.concat([concatenated_df, col_comparison], axis=1)
        
    return concatenated_df

In [55]:
concat_described(data_frames = [wines.where(wines["wine_type"] == "red"), wines.where(wines["wine_type"] == "white")], 
                 subset_attributes=["residual sugar", "total sulfur dioxide", "sulphates", "alcohol", "volatile acidity", "quality"],
                 keys=["red", "white"])

,red residual sugar,white residual sugar,red total sulfur dioxide,white total sulfur dioxide,red sulphates,white sulphates,red alcohol,white alcohol,red volatile acidity,white volatile acidity,red quality,white quality
mean,2.538806,6.391415,46.467792,138.360657,0.658149,0.489847,10.422983,10.514267,0.527821,0.278241,5.636023,5.877909
std,1.409928,5.072058,32.895324,42.498065,0.169507,0.114126,1.065668,1.230621,0.179060,0.100795,0.807569,0.885639
min,0.900000,0.600000,6.000000,9.000000,0.330000,0.220000,8.400000,8.000000,0.120000,0.080000,3.000000,3.000000
25%,1.900000,1.700000,22.000000,108.000000,0.550000,0.410000,9.500000,9.500000,0.390000,0.210000,5.000000,5.000000
50%,2.200000,5.200000,38.000000,134.000000,0.620000,0.470000,10.200000,10.400000,0.520000,0.260000,6.000000,6.000000
75%,2.600000,9.900000,62.000000,167.000000,0.730000,0.550000,11.100000,11.400000,0.640000,0.320000,6.000000,6.000000
max,15.500000,65.800000,289.000000,440.000000,2.000000,1.080000,14.900000,14.200000,1.580000,1.100000,8.000000,9.000000


In [59]:
concat_described(data_frames = [wines.where(wines["quality_label"] == "low"), 
                                wines.where(wines["quality_label"] == "medium"),
                                wines.where(wines["quality_label"] == "high")], 
                 subset_attributes=["alcohol", "volatile acidity", "pH", "quality"],
                 keys=["q low", "q medium", "q high"],
                 drop_count=False)

,q low alcohol,q medium alcohol,q high alcohol,q low volatile acidity,q medium volatile acidity,q high volatile acidity,q low pH,q medium pH,q high pH,q low quality,q medium quality,q high quality
count,246.000000,4974.000000,1277.000000,246.000000,4974.000000,1277.00000,246.000000,4974.000000,1277.000000,246.000000,4974.000000,1277.000000
mean,10.184350,10.265275,11.433359,0.465163,0.346423,0.28917,3.234797,3.215346,3.227651,3.878049,5.570165,7.158966
std,0.999035,1.070626,1.215620,0.245677,0.165660,0.11696,0.191313,0.159461,0.159094,0.327897,0.495102,0.376348
min,8.000000,8.000000,8.500000,0.110000,0.080000,0.08000,2.740000,2.720000,2.840000,3.000000,5.000000,7.000000
25%,9.400000,9.400000,10.700000,0.280000,0.230000,0.20000,3.090000,3.110000,3.120000,4.000000,5.000000,7.000000
50%,10.050000,10.000000,11.500000,0.380000,0.300000,0.27000,3.225000,3.200000,3.220000,4.000000,6.000000,7.000000
75%,10.900000,11.000000,12.400000,0.610000,0.420000,0.34000,3.360000,3.320000,3.340000,4.000000,6.000000,7.000000
max,13.500000,14.900000,14.200000,1.580000,1.330000,0.91500,3.900000,4.010000,3.820000,4.000000,6.000000,9.000000
